In [ ]:
#Pregunta numero 1
install.packages("hdm")
install.packages("xtable")
install.packages("hdm")
install.packages("xtable")

In [ ]:
# Cargar los datos del censo 2015
file <- "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/wage2015_subsample_inference.csv"
data <- read.csv(file)

# Ver estructura inicial de los datos
str(data)

# Variable dependiente 
y <- data$lwage

# Crear matriz de variables explicativas con interacciones bidireccionales
Z <- subset(data, select = -c(lwage, wage))

# Centrar columnas le quitamos la media
center_colmeans <- function(x) {
  xcenter <- colMeans(x)
  x - rep(xcenter, rep.int(nrow(x), ncol(x)))
}

# Fórmula para controles bidireccionales, construye las categorias posbiles
controls_formula <- "~ 0 + (shs + hsg + scl + clg + C(occ2) + C(ind2) + mw + so + we + exp1 + exp2 + exp3 + exp4)**2"
Zcontrols <- model.matrix(as.formula(controls_formula), data = Z)
Zcontrols <- center_colmeans(Zcontrols)

# Crea interacciones con heterogeneidad lineal y nos quedamos con las columnas
linear_het_formula <- "~ -1 + (shs + hsg + scl + clg + mw + so + we + exp1 + exp2 + exp3 + exp4)"
Zhet <- model.matrix(as.formula(linear_het_formula), data = Z)
Zhet <- center_colmeans(Zhet)

# Incluir interacciones de género con heterogeneidad
Zhet <- as.data.frame(cbind(Zhet, "sex" = Z$sex))
nonlin_het_formula <- "~ -1 + sex + sex * (shs + hsg + scl + clg + mw + so + we + exp1 + exp2 + exp3 + exp4)"
Zinteractions <- model.matrix(as.formula(nonlin_het_formula), data = Zhet)

# Combinar todas las variables
interaction_cols <- Zinteractions[, grepl("sex", colnames(Zinteractions))]
X <- cbind(Zinteractions, Zcontrols)

# Índice de variables de interacción con género
index_gender <- grep("sex", colnames(Zinteractions))

# Aplicar el modelo de doble lazo
set.seed(123)
effects_female <- hdm::rlassoEffects(x = X, y = y, index = index_gender, post = FALSE)

# Resumen de los resultados
result <- summary(effects_female)
print(result$coef)

# Intervalos de confianza punto a punto
pointwise_ci <- confint(effects_female, level = 0.95)
print(pointwise_ci)

# Intervalos de confianza conjuntos
joint_ci <- confint(effects_female, level = 0.95, joint = TRUE)
print(joint_ci)



library(dplyr)
library(caret)
library(glmnet)

data_url <- "https://raw.githubusercontent.com/d2cml-ai/CausalAI-Course/main/data/wage2015_subsample_inference.csv"
df <- read.csv(data_url)
str(df)
# Variables de interés
variables_categoricas <- c("sex", "mw", "so", "we", "ne")
variables_numericas <- c("wage", "experience", "occupation", "marital_status")  
# Ajustar según datos reales

# Crear interacciones y convertir categóricas en dummies
dataset_preparado <- census_data_2015 %>%
  mutate(across(all_of(variables_categoricas), ~ as.factor(.))) %>%
  model.matrix(~ .^2, data = .) %>%  # Genera todas las interacciones de orden 2
  as.data.frame() %>%
  select(-(Intercept))  # Remover la constante añadida por model.matrix

# Verificar el dataset
str(dataset_preparado)
ibrary(ggdag)
dag_smoking <- dagify(
  lung_function ~ smoking + age + height + sex,
  smoking ~ age + sex,
  exposure = "smoking",
  outcome = "lung_function"
)

ggdag(dag_smoking) + theme_dag()
dag_breastfeeding <- dagify(
  infections ~ breastfeeding + marital_status + family_income + education + childcare,
  breastfeeding ~ marital_status + family_income + education,
  exposure = "breastfeeding",
  outcome = "infections"
)

ggdag(dag_breastfeeding) + theme_dag()


#Pregunta 1 parte 2
#install.packages("dagitty")
#install.packages("ggdag")
library(dagitty)
library(ggdag)
# Crear el DAG
#Ejemplo 1: Efecto del tabaquismo juvenil en la función pulmonar
#Variables:
#Tratamiento: Conducta  de tabaquismo.
#Resultado: Volumen respiratorio forzado.
#Confusores potenciales: Edad y  Sexo.

dag_smoking <- dagitty('
dag {
  Smoking -> LungFunction
  Age -> Smoking
  Age -> LungFunction
  Height -> LungFunction
  Sex -> Smoking
  Sex -> LungFunction
}')

# Dibujar el DAG
plot(dag_smoking)

# Visualización más moderna con ggdag
tidy_dag_smoking <- tidy_dagitty(dag_smoking)
ggdag(tidy_dag_smoking) +
  theme_minimal() +
  ggtitle("DAG: Efecto del tabaquismo juvenil en la función pulmonar")

#Confusores: Edad y Sexo (influyen tanto en el tabaquismo como en la función pulmonar).
#Controles adecuados: Edad y Sexo deben incluirse como controles en el modelo para evitar sesgos."

#Ejemplo 2: Efecto de la lactancia materna en infecciones del bebé

#Tratamiento: Lactancia materna.
#Resultado: Número de infecciones en bebes.
#Confusores potenciales: Estado civil, Ingresos familiares, Educación.
#Otros factores relacionados: Número de niños en el hogar, Cuidado de niños fuera del hogar."
# Crear el DAG
dag_breastfeeding <- dagitty('
dag {
  Breastfeeding -> Infections
  MaritalStatus -> Breastfeeding
  MaritalStatus -> Infections
  FamilyIncome -> Breastfeeding
  FamilyIncome -> Infections
  Education -> Breastfeeding
  Education -> Infections
  Breastfeeding -> Childcare
  Breastfeeding -> NumberOfChildren
  Infections -> Childcare
  Infections -> NumberOfChildren
}')

# Dibujar el DAG
plot(dag_breastfeeding)

# Visualización con ggdag
tidy_dag_breastfeeding <- tidy_dagitty(dag_breastfeeding)
ggdag(tidy_dag_breastfeeding) +
  theme_minimal() +
  ggtitle("DAG: Efecto de la lactancia materna en infecciones del bebé")

# Pregunta 2
install.packages("caret")
install.packages("dplyr")
install.packages("readr")
install.packages("fastDummies")

# Cargar las librerías
library(dplyr)
library(caret)
library(readr)
library(fastDummies)

# Cargar los datos desde la URL o ruta
data_url <- "https://raw.githubusercontent.com/d2cml-ai/CausalAI-Course/main/data/wage2015_subsample_inference.csv"
df <- read.csv(data_url)

# Revisar la estructura de los datos
str(df)

# Asegurarse de que 'occ2' e 'ind2' sean factores
df$occ2 <- as.factor(df
ind2 <- as.factor(df$ind2)

# Crear las variables de experiencia
extra_flexible_model_vars <- c('exp1', 'exp2', 'exp3', 'exp4', 'hsg', 'scl', 'clg', 'ad', 'so', 'we', 'ne')

# Generar variables dummy para occ2 e ind2
df_with_dummies <- dummy_cols(df, select_columns = c("occ2", "ind2"), remove_first_dummy = TRUE)

# Extraer las columnas dummy
dummy_vars <- names(df_with_dummies)[grepl("occ2_|ind2_", names(df_with_dummies))]

# Crear un vector con todas las variables para el modelo
all_vars <- c(extra_flexible_model_vars, dummy_vars)

# Generar interacciones de dos vías
interactions <- NULL
for (i in 1:(length(all_vars) - 1)) {
  for (j in (i + 1):length(all_vars)) {
    interaction_var <- df_with_dummies[[all_vars[i]]] * df_with_dummies[[all_vars[j]]]
    interactions <- cbind(interactions, interaction_var)
  }
}

# Combinar las variables originales y las interacciones
extra_flexible_model_array <- cbind(df_with_dummies[, all_vars], interactions)

# Extraer y normalizar la variable Y (lwage)
log_w <- as.matrix(df$lwage)

normalize_array <- function(arr) {
  min_vals <- apply(arr, 2, min)
  max_vals <- apply(arr, 2, max)
  range_vals <- max_vals - min_vals
  range_vals[range_vals == 0] <- 1
  norm_arr <- (arr - min_vals) / range_vals
  return(norm_arr)
}

# Normalizar Y
norm_log_w <- normalize_array(log_w)

# Normalizar las variables predictoras X
x_data <- normalize_array(extra_flexible_model_array)

install.packages("caTools")
library(caTools)

set.seed(123)  # Para reproducibilidad
split <- sample.split(norm_log_w, SplitRatio = 0.9)

# Dividir los datos
x_train <- x_data[split, ]
x_test <- x_data[!split, ]
y_train <- norm_log_w[split, ]
y_test <- norm_log_w[!split, ]


# Función para generar una grilla logarítmica
generate_log_grid <- function(lower_bound, upper_bound, spacing) {
  grid <- exp(seq(log(lower_bound), log(upper_bound), length.out = spacing))
  return(grid)
}

# Función para generar k folds
generate_k_folds <- function(data, k) {
  folds <- sample(rep(1:k, length.out = nrow(data)))
  return(folds)
}

# Función para validar Lasso y encontrar el lambda óptimo
find_optimal_lambda <- function(Y, X, lambda_bounds, k) {
  lambdas <- generate_log_grid(lambda_bounds[1], lambda_bounds[2], 100)
  folds <- generate_k_folds(X, k)
  mse_per_lambda <- c()
  
  for (lambda in lambdas) {
    mse_fold <- c()
    
    for (fold in 1:k) {
      train_indices <- which(folds != fold)
      test_indices <- which(folds == fold)
      
      X_train <- X[train_indices, ]
      Y_train <- Y[train_indices, ]
      X_test <- X[test_indices, ]
      Y_test <- Y[test_indices, ]
      
      # Entrenar modelo de Lasso (usando glmnet como ejemplo)
      lasso_model <- glmnet(X_train, Y_train, alpha = 1, lambda = lambda)
      predictions <- predict(lasso_model, newx = X_test)
      mse <- mean((Y_test - predictions)^2)
      mse_fold <- c(mse_fold, mse)
    }
    
    mse_per_lambda <- c(mse_per_lambda, mean(mse_fold))
  }
  
  # Encontrar el lambda con menor MSE
  optimal_lambda <- lambdas[which.min(mse_per_lambda)]
  optimal_model <- glmnet(X, Y, alpha = 1, lambda = optimal_lambda)
  optimal_coef <- coef(optimal_model)
  
  return(list(optimal_lambda = optimal_lambda, 
              optimal_coef = optimal_coef, 
              all_lambdas = lambdas, 
              all_mse = mse_per_lambda))
}


# Función para predecir con el modelo óptimo
predict_with_optimal_lambda <- function(optimal_model, X) {
  predictions <- predict(optimal_model, newx = X)
  return(predictions)
}


# Ajustar un modelo OLS simple con la muestra de entrenamiento
ols_model <- lm(y_train ~ x_train)

# Mostrar el resumen del modelo OLS
summary(ols_model)

# Definir los límites de la grilla de lambdas y el número de folds
lambda_bounds <- c(exp(-7), exp(7))
k_folds <- 5

# Encontrar el lambda óptimo y los coeficientes con la función programada
optimal_lasso <- find_optimal_lambda(Y = y_train, X = x_train, lambda_bounds = lambda_bounds, k = k_folds)

# Imprimir el lambda óptimo y los coeficientes asociados
cat("Lambda óptimo:", optimal_lasso$optimal_lambda, "\n")
cat("Coeficientes asociados al lambda óptimo:\n", optimal_lasso$optimal_coef)


install.packages("hdm")
library(hdm)

# Ajustar el modelo Lasso con el lambda óptimo usando el paquete hdm
lasso_hdm <- rlasso(y_train ~ x_train, post = FALSE, lambda = optimal_lasso$optimal_lambda)

# Mostrar los resultados del modelo Lasso ajustado con hdm
summary(lasso_hdm)

# Predecir con el modelo OLS en el conjunto de prueba
ols_predictions <- predict(ols_model, newdata = data.frame(x_test))

# Calcular el MSE para el modelo OLS
mse_ols <- mean((y_test - ols_predictions)^2)

# Calcular el R² para el modelo OLS
r2_ols <- 1 - sum((y_test - ols_predictions)^2) / sum((y_test - mean(y_test))^2)

cat("MSE para el modelo OLS:", mse_ols, "\n")
cat("R² para el modelo OLS:", r2_ols, "\n")

# Predecir con el modelo Lasso de validación cruzada
lasso_predictions <- predict_with_optimal_lambda(optimal_lasso, x_test)

# Calcular el MSE para el modelo Lasso con validación cruzada
mse_lasso <- mean((y_test - lasso_predictions)^2)

# Calcular el R² para el modelo Lasso con validación cruzada
r2_lasso <- 1 - sum((y_test - lasso_predictions)^2) / sum((y_test - mean(y_test))^2)

cat("MSE para el modelo Lasso con validación cruzada:", mse_lasso, "\n")
cat("R² para el modelo Lasso con validación cruzada:", r2_lasso, "\n")

# Predecir con el modelo Lasso de hdm en el conjunto de prueba
hdm_predictions <- predict(lasso_hdm, newdata = data.frame(x_test))

# Calcular el MSE para el modelo Lasso con hdm
mse_hdm <- mean((y_test - hdm_predictions)^2)

# Calcular el R² para el modelo Lasso con hdm
r2_hdm <- 1 - sum((y_test - hdm_predictions)^2) / sum((y_test - mean(y_test))^2)

cat("MSE para el modelo Lasso con hdm:", mse_hdm, "\n")
cat("R² para el modelo Lasso con hdm:", r2_hdm, "\n")